# Fair Binary Classification with SearchFair on CelebA and Adult

Here, we show how to use SearchFair on two datasets: CelebA and Adult

## Imports

We start by importing SearchFair from the installed package.

In [5]:
from searchfair import SearchFair
import pandas as pd

Second, we load some necessary methods and numpy.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np

# The optimization does not always comply with the new cvxpy dpp disciplined programming rules. 
# but this is not a problem. 
import warnings
warnings.filterwarnings('ignore')

# The CelebA dataset

On the Celebrity Faces dataset we are given descriptions of celebrity faces, with 40 binary attributes. Here, we use the Attribute 'Smiling' as the class label, and sex as the sensitive attribute. 

In [7]:
!Ls

Data Cleaning Recidivism.ipynb real_data.ipynb
__pycache__                    recividism_cleaned.csv
get_real_data.py               toy_data.ipynb
get_synthetic_data.py          utils.py
real_data - recidivism.ipynb


In [ ]:
# importing cleaned recidivism data from IOWA
# https://data.iowa.gov/Correctional-System/3-Year-Recidivism-for-Offenders-Released-from-Pris/mw8r-vqy4

In [11]:
df = pd.read_csv('recividism_cleaned.csv')

In [12]:
df

,Release Type_Discharged - Expiration of Sentence,Release Type_Discharged – End of Sentence,Release Type_Parole,Release Type_Parole Granted,Release Type_Paroled to Detainer - INS,Release Type_Paroled to Detainer - Iowa,Release Type_Paroled to Detainer - Out of State,Release Type_Paroled to Detainer - U.S. Marshall,Release Type_Paroled w/Immediate Discharge,Release Type_Released to Special Sentence,...,Offense Subtype_Sex Offender Registry/Residency,Offense Subtype_Special Sentence Revocation,Offense Subtype_Stolen Property,Offense Subtype_Theft,Offense Subtype_Traffic,Offense Subtype_Trafficking,Offense Subtype_Vandalism,Offense Subtype_Weapons,Race,Recidivism
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,-1,1
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,-1,1
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,-1
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21045,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,-1,-1
21046,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,-1
21047,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,-1,-1
21048,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,-1,-1


In [13]:
s_data = df.Race.values

In [22]:
s_data.shape

(21050,)

In [14]:
y_data = df.Recidivism.values

In [23]:
x_data = df[df.columns[:-2]].values

In [24]:
len(x_values)

21050

In [25]:
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)

In [3]:
import get_real_data as get_data

# Load Data
x_data, y_data, s_data = get_data.get_celebA_data(load_data_size=None)
# Train Test split. Here, we choose a small number to reduce running time.
train_size = 1200
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)

Here are some basic information about the dataset:

In [13]:
x_data[:10][0]

array([-1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,
       -1,  1,  1,  1, -1, -1,  1, -1, -1,  1, -1, -1, -1,  1, -1,  1, -1,
        1, -1, -1,  1])

In [14]:
len(y_train)

1200

In [9]:
y_train[:10]

array([-1,  1,  1, -1,  1, -1,  1, -1,  1, -1])

In [8]:
s_train[:10]

array([ 1, -1,  1, -1,  1, -1,  1,  1,  1,  1], dtype=int64)

In [26]:
import utils as ut
ut.print_data_stats(s_data, y_data)

Total data points: 21050
# non-protected examples: 5129
# protected examples: 15921
# non-protected examples in positive class: 1797 (35.0%)
# protected examples in positive class: 5569 (35.0%)


## Creating a SearchFair Model

### Demographic Parity

To learn a classifier with SearchFair, we need to choose a kernel between 'linear' and 'rbf', and we need to choose a fairness notion - either Demographic Parity (DDP) or Equality of Opportunity (DEO). 

In [27]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'linear' # 'linear', 'rbf'
verbose = True # True = SearchFair output, 2 = show also solver progress

Let us choose a regularization parameter and then fit a model with the default settings. 

In [28]:
# Regularization Parameter beta
reg_beta = 0.0001
linear_model_DDP = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose, stop_criterion=0.01)
linear_model_DDP.fit(x_train, y_train, s_train=s_train)

Preprocessing...
Testing lambda_min: 0.00
Obtained: DDP = -0.0217 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DDP = 0.1942 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DDP = 0.1920 with lambda = 0.5000
----------Iteration #1----------
Testing new Lambda: 0.2500
Obtained: DDP = 0.0969 with lambda = 0.2500
----------Iteration #2----------
Testing new Lambda: 0.1250
Obtained: DDP = 0.0693 with lambda = 0.1250
----------Iteration #3----------
Testing new Lambda: 0.0625
Obtained: DDP = -0.0217 with lambda = 0.0625
----------Iteration #4----------
Testing new Lambda: 0.0938
Obtained: DDP = 0.0599 with lambda = 0.0938
----------Iteration #5----------
Testing new Lambda: 0.0781
Obtained: DDP = -0.0217 with lambda = 0.0781
----------Iteration #6----------
Testing new Lambda: 0.0859
Obtained: DDP = -0.0217 with lambda = 0.0859
----------Iteration #7----------
Testing new Lambda: 0.0898
Obtained: DDP = -0.0217 wit

SearchFair(reg_beta=0.0001, verbose=True)

To print out the Accuracy and the fairness notions Demographic Parity and Equality of Opportuniy, we define the following function. 

In [29]:
def print_clf_stats(model, x_train, x_test, y_train, y_test, s_train, s_test):
    train_acc = ut.get_accuracy(np.sign(model.predict(x_train)), y_train)
    test_acc = ut.get_accuracy(np.sign(model.predict(x_test)), y_test)
    test_DDP, test_DEO = ut.compute_fairness_measures(model.predict(x_test), y_test, s_test)
    train_DDP, train_DEO = ut.compute_fairness_measures(model.predict(x_train), y_train, s_train)

    print(10*'-'+"Train"+10*'-')
    print("Accuracy: %0.4f%%" % (train_acc * 100))
    print("DDP: %0.4f%%" % (train_DDP * 100), "DEO: %0.4f%%" % (train_DEO * 100))
    print(10*'-'+"Test"+10*'-')
    print("Accuracy: %0.4f%%" % (test_acc * 100))
    print("DDP: %0.4f%%" % (test_DDP * 100), "DEO: %0.4f%%" % (test_DEO * 100))

Now lets see, if we obtained a fair classifier with respect to the fairness notions we specified. 

In [30]:
print_clf_stats(linear_model_DDP, x_train, x_test, y_train, y_test, s_train, s_test)

----------Train----------
Accuracy: 64.8333%
DDP: -2.1743% DEO: -3.2929%
----------Test----------
Accuracy: 65.1637%
DDP: -0.6283% DEO: -0.7367%


The train DDP is small, and SearchFair succeeded to find a fair classifier. The test DDP might or might not be close to 0. This is due to the small number of points which are used to reduce running time for this example notebook. Go ahead and try it with more points!

### Equality of Opportunity

Now we try the same using a more complex rbf kernel, and we try to improve Equality of Opportunity this time. 

In [15]:
fairness_notion = 'DEO' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = True

# Regularization Parameter beta
reg_beta = 0.0001
rbf_model_DEO = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)
rbf_model_DEO.fit(x_train, y_train, s_train=s_train)

# Evaluate model
print_clf_stats(rbf_model_DEO, x_train, x_test, y_train, y_test, s_train, s_test)

Preprocessing...
Testing lambda_min: 0.00
Obtained: DEO = 0.1603 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DEO = -0.8886 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DEO = -0.7966 with lambda = 0.5000
----------Iteration #1----------
Testing new Lambda: 0.2500
Obtained: DEO = -0.1373 with lambda = 0.2500
----------Iteration #2----------
Testing new Lambda: 0.1250
Obtained: DEO = 0.0951 with lambda = 0.1250
----------Iteration #3----------
Testing new Lambda: 0.1875
Obtained: DEO = -0.0153 with lambda = 0.1875
----------Iteration #4----------
Testing new Lambda: 0.1562
Obtained: DEO = 0.0851 with lambda = 0.1562
----------Iteration #5----------
Testing new Lambda: 0.1719
Obtained: DEO = 0.0579 with lambda = 0.1719
----------Iteration #6----------
Testing new Lambda: 0.1797
Obtained: DEO = 0.0279 with lambda = 0.1797
----------Iteration #7----------
Testing new Lambda: 0.1836
Obtained: DEO = 0.0168 with

### Cross Validation - GridSearch

It is also possible to use GridSearchCV for the regularization paramter beta, and, if used, the width of the rbf kernel. But running this might take some time.

In [ ]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = False

cv_model = SearchFair(kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)

# regularization parameter beta
beta_params = [0.0001, 0.001, 0.01]
cv_params = {'reg_beta': beta_params}

if kernel == 'rbf':
    n_features = x_data.shape[1]
    default_width = 1/n_features
    order_of_magn = np.floor(np.log10(default_width))
    kernel_widths = [10**(order_of_magn), default_width, 10**(order_of_magn+1)]
    cv_params['gamma'] = kernel_widths

grid_clf = GridSearchCV(cv_model,cv_params, cv=3, verbose=1, n_jobs=1, scoring='accuracy', refit=True)
grid_clf.fit(x_train, y_train, s_train=s_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000


In [ ]:
# Evaluate model
print_clf_stats(grid_clf, x_train, x_test, y_train, y_test, s_train, s_test)

## Adult dataset

In the fairness literature, the adult dataset is a very popular dataset. It contains US census data from 1994, where the class label indicates if the income is higher or lower than 50.000$. The binary sensitive attribute here, is the sex.

In [ ]:
# Load Data
x_data, y_data, s_data = get_data.get_adult_data(load_data_size=None)
# Train Test split. Here, we choose a small number to reduce running time.
train_size = 1200
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)
ut.print_data_stats(s_data, y_data)

If you want, you can also try SearchFair on Adult. 

In [ ]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = False

cv_model_adult = SearchFair(kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)

# regularization parameter beta
beta_params = [0.0001, 0.001, 0.01]
cv_params = {'reg_beta': beta_params}

if kernel == 'rbf':
    n_features = x_data.shape[1]
    default_width = 1/n_features
    order_of_magn = np.floor(np.log10(default_width))
    kernel_widths = [10**(order_of_magn), default_width, 10**(order_of_magn+1)]
    cv_params['gamma'] = kernel_widths

grid_clf = GridSearchCV(cv_model_adult,cv_params, cv=3, verbose=1, n_jobs=1, scoring='accuracy')
grid_clf.fit(x_train, y_train, s_train=s_train)

In [ ]:
# Evaluate model
print_clf_stats(grid_clf, x_train, x_test, y_train, y_test, s_train, s_test)